In [ ]:
import json
import csv
import os
from datetime import datetime

# The directory containing your folders with JSON files
root_directory = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\track_information'

# The path for the output CSV file
csv_file_path = 'tracks_data.csv'

# Current date for 'Date Added To DB' and 'Date Last Modified' columns in the YYYY-MM-DD format
current_date = datetime.now().strftime('%Y-%m-%d')

# Template for CSV rows with the updated header titles
csv_columns = [
    'id', 'song_spotify_id', 'song_title', 'song_duration', 'song_album_type', 
    'song_album_id', 'song_explicit', 'song_popularity', 'song_preview_url', 
    'song_track_features_added', 'song_acousticness', 'song_danceability', 'song_energy', 
    'song_instrumentalness', 'song_liveness', 'song_loudness', 'song_speechiness', 
    'song_tempo', 'song_valence', 'song_key', 'song_time_signature', 'song_date_added_to_db', 
    'song_date_last_modified'
]

# Initialize an empty list to store rows for the CSV
csv_rows = []

def process_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        for track in data.get('tracks', []):
            # Skip processing if the track is None
            if track is None:
                continue

            row = {
                'id': -1,  # This will be updated later with the actual ID
                'song_spotify_id': track['id'],
                'song_title': track['name'],
                'song_duration': track['duration_ms'],
                'song_album_type': track['album']['album_type'].upper(),
                'song_album_id': track['album']['id'],
                'song_explicit': track['explicit'],
                'song_popularity': track['popularity'],
                'song_preview_url': track.get('preview_url', ''),
                'song_track_features_added': False,
                'song_acousticness': -1,
                'song_danceability': -1,
                'song_energy': -1,
                'song_instrumentalness': -1,
                'song_liveness': -1,
                'song_loudness': -1,
                'song_speechiness': -1,
                'song_tempo': -1,
                'song_valence': -1,
                'song_key': -1,
                'song_time_signature': -1,
                'song_date_added_to_db': current_date,
                'song_date_last_modified': current_date
            }
            csv_rows.append(row)

# Iterate over each subfolder and JSON file in the directory
for subdir, dirs, files in os.walk(root_directory):
    for filename in files:
        if filename.endswith('.json'):
            print(f"Processing {filename}...")
            process_json_file(os.path.join(subdir, filename))

# Write the CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns, delimiter=';')
    writer.writeheader()
    
    # Update each row with its ID before writing
    for i, row in enumerate(csv_rows, start=1):
        row['id'] = i
        writer.writerow(row)

print(f"CSV file has been successfully created at {csv_file_path} with {len(csv_rows)} tracks.")


In [ ]:
import json
import csv
import os
from datetime import datetime

# Define the root directory containing your JSON files
root_directory = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\ALBUMS'

# Define the output CSV file path
csv_file_path = 'album_data.csv'

# Current date for 'Date Added To DB' and 'Date Last Modified' columns
current_date = datetime.now().strftime('%Y-%m-%d')

# CSV column headers
csv_columns = [
    'id', 'album_spotify_id', 'album_name', 'album_cover_art', 'album_release_date',
    'release_date_precision', 'album_popularity', 'album_type', 'spotify_album_upc',
    'spotify_album_ean', 'spotify_album_isrc', 'date_added_to_db', 'date_last_modified',
    'musicbrainz_metadata_added', 'musicbrainz_id'
]

# Initialize a list to hold album data
albums_data = []

# Function to process each JSON file
def process_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        for album in data.get('albums', []):
            # Extract the required information, with checks for nullable fields
            album_data = {
                'id': -1,  # Placeholder, will be updated later
                'album_spotify_id': album['id'],
                'album_name': album['name'],
                'album_cover_art': album['images'][0]['url'] if album.get('images') else '',
                'album_release_date': album['release_date'],
                'release_date_precision': album['release_date_precision'],
                'album_popularity': album['popularity'],
                'album_type': album['album_type'],
                'spotify_album_upc': album['external_ids'].get('upc', '') if album.get('external_ids') else '',
                'spotify_album_ean': album['external_ids'].get('ean', '') if album.get('external_ids') else '',
                'spotify_album_isrc': album['external_ids'].get('isrc', '') if album.get('external_ids') else '',
                'date_added_to_db': current_date,
                'date_last_modified': current_date,
                'musicbrainz_metadata_added': False,  # Placeholder
                'musicbrainz_id': ''  # Placeholder
            }
            albums_data.append(album_data)

# Process each JSON file in the directory and subdirectories
for subdir, dirs, files in os.walk(root_directory):
    for filename in files:
        if filename.endswith('.json'):
            process_json_file(os.path.join(subdir, filename))

# Write the data to a CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns, delimiter=';')
    writer.writeheader()
    
    # Update each row with its actual ID before writing
    for i, album_data in enumerate(albums_data, start=1):
        album_data['id'] = i
        writer.writerow(album_data)

print(f"CSV file has been successfully created at {csv_file_path} with {len(albums_data)} albums.")


In [ ]:
import pandas as pd

# Specify the dtypes for the IDs to be strings when reading the CSVs
dtype_dict = {'id': str, 'song_album_id': str, 'album_spotify_id': str}
tracks_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\tracks_data.csv', delimiter=';', dtype=dtype_dict)
albums_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\album_data.csv', delimiter=';', dtype=dtype_dict)

# Create a dictionary mapping from album_spotify_id to id from albums_df
# Ensure the 'id' column in albums_df is converted to integer if it's not NaN
album_id_map = albums_df.dropna(subset=['id']).set_index('album_spotify_id')['id'].astype(int).to_dict()

# Map the song_album_id in tracks_df using the album_id_map to get the album id
tracks_df['album_id'] = tracks_df['song_album_id'].map(album_id_map)

# Convert the new album_id column to integers, NaNs will be converted to a float with a .0
tracks_df['album_id'] = tracks_df['album_id'].fillna(-1).astype(int)

# Replace -1 back to NaN if you want to keep NaN values
tracks_df['album_id'].replace(-1, pd.NA, inplace=True)

# Save the updated tracks DataFrame to a new CSV file
tracks_df.to_csv('path_to_updated_tracks.csv', index=False, sep=';')


In [ ]:



import pandas as pd
import os
import json

# Read the albums and artists CSVs into DataFrames
albums_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\album_data_full.csv', delimiter=';', dtype={'id': str, 'album_spotify_id': str})
artists_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\artists_data_full.csv', delimiter=';', dtype={'id': str, 'artist_spotify_id': str})

# Dictionary to map Spotify album ID to CSV album ID
album_id_map = albums_df.set_index('album_spotify_id')['id'].to_dict()

# Dictionary to map Spotify artist ID to CSV artist ID
artist_id_map = artists_df.set_index('artist_spotify_id')['id'].to_dict()

# Initialize a list to hold the artist-album mappings
artist_album_mapping = []

# Assuming 'path_to_json_folder' is the folder containing all the JSON subfolders
for root, dirs, files in os.walk('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\ALBUMS'):
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                data = json.load(f)
                for album in data['albums']:
                    album_id = album_id_map.get(album['id'])
                    if album_id:
                        for artist in album['artists']:
                            artist_id = artist_id_map.get(artist['id'])
                            if artist_id:
                                artist_album_mapping.append({'artistID': artist_id, 'albumID': album_id})

# Create a DataFrame from the artist-album mappings
artist_album_df = pd.DataFrame(artist_album_mapping)

# Remove duplicates if there are any
artist_album_df = artist_album_df.drop_duplicates()

# Save the DataFrame to CSV
artist_album_df.to_csv('artist_album_mappings_new.csv', index=False, sep=';')


In [ ]:
import pandas as pd
import os
import json

# Read the albums and artists CSVs into DataFrames
albums_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\album_data.csv', delimiter=';', dtype={'id': str, 'album_spotify_id': str})
artists_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\artists_data_full.csv', delimiter=';', dtype={'id': str, 'artist_spotify_id': str})

# Dictionary to map Spotify album ID to CSV album ID
album_id_map = albums_df.set_index('album_spotify_id')['id'].to_dict()

# Dictionary to map Spotify artist ID to CSV artist ID
artist_id_map = artists_df.set_index('artist_spotify_id')['id'].to_dict()

# Initialize a list to hold the artist-album mappings
artist_album_mapping = []

# List to hold artist Spotify IDs where CSV artist ID was not found
missing_artist_ids = []

# Assuming 'path_to_json_folder' is the folder containing all the JSON subfolders
for root, dirs, files in os.walk('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\ALBUMS'):
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                data = json.load(f)
                for album in data['albums']:
                    album_id = album_id_map.get(album['id'])
                    if album_id:
                        for artist in album['artists']:
                            artist_spotify_id = artist['id']
                            artist_id = artist_id_map.get(artist_spotify_id)
                            if artist_id:
                                artist_album_mapping.append({'artistID': artist_id, 'albumID': album_id})
                            else:
                                missing_artist_ids.append(artist_spotify_id)

# Print the list of artist Spotify IDs where the CSV artist ID was not found
print("List of artist Spotify IDs where the CSV artist ID was not found:")
for artist_id in missing_artist_ids:
    print(artist_id)

In [ ]:
import pandas as pd

# Convert the list to a DataFrame
missing_artist_ids_df = pd.DataFrame(missing_artist_ids, columns=['missing_artist_spotify_id'])

# Define the file path where you want to save the CSV
file_path = 'missing_artist_ids.csv'  # You can specify your own path

# Save the DataFrame to a CSV file
missing_artist_ids_df.to_csv(file_path, index=False)

print(f'The missing artist IDs have been saved to {file_path}')

In [ ]:
import pandas as pd

# Replace 'input_file.csv' with the path to your input CSV file
input_file_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\musicBrainzAllContributors.csv'
# Replace 'output_file.csv' with the path where you want to save the output CSV file
output_file_path = 'contributor_data_prelink.csv'

# Step 1: Read the input CSV
input_df = pd.read_csv(input_file_path, delimiter=',')

# Step 2: Create the new DataFrame
output_df = pd.DataFrame({
    'ID': range(1, len(input_df) + 1),
    'NAME': input_df['artist_credit_name'],
    'ROLE': input_df['role'],
    'INSTRUMENT': input_df['instrument'],
    'MUSICBRAINZ_ID': input_df['artist_mbid'],
    'MAINARTIST': input_df['artist_credit_name'],
    'SONGTITLE': input_df['recording_name']
})

# Step 3: Write the resulting DataFrame to a new CSV file
output_df.to_csv(output_file_path, sep=';', index=False)

print(f"Output CSV saved to {output_file_path}")


In [ ]:
import pandas as pd
from fuzzywuzzy import process

# Load the CSV files
musicbrainz_df = pd.read_csv('C:\\Users\\Music\\team_project\\team37\\prepopulationStuff\\PythonNotebooksForPrepopulation\\contributor_data_prelink.csv', sep=';')  # The file generated from the previous step
spotify_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\tracks_data.csv', sep=';')  # The Spotify songs CSV

# Create a list of combined song title and artist names from Spotify for matching
spotify_combined_list = spotify_df.apply(lambda x: f"{x['song_title']} {x['song_album_id']}", axis=1).tolist()

def find_best_match(mb_row, spotify_combined_list):
    mb_combined = f"{mb_row['SONGTITLE']} {mb_row['MAINARTIST']}"
    
    # Using extractOne to find the best match from the list
    best_match_info = process.extractOne(mb_combined, spotify_combined_list)
    
    # If there is a match found, extract it
    if best_match_info:
        best_match_text, best_score = best_match_info
        # Find the index of the match in Spotify list to retrieve the full row from spotify_df
        match_index = spotify_combined_list.index(best_match_text)
        best_match_row = spotify_df.iloc[match_index]
        return best_match_row, best_score
    return None, 0

# Prepare the output DataFrame
matched_df = pd.DataFrame(columns=['CONTRIBUTOR_ID', 'SONG_TABLE_ID', 'Spotify_Song_Title', 'Spotify_Artist', 'MusicBrainz_Song_Title', 'MusicBrainz_Artist'])

# Iterate over MusicBrainz entries to find matches
for index, mb_row in musicbrainz_df.iterrows():
    best_match_row, score = find_best_match(mb_row, spotify_combined_list)
    if best_match_row is not None and score > 80:  # Adjust the threshold as needed
        matched_df = matched_df.append({
            'CONTRIBUTOR_ID': mb_row['ID'],
            'SONG_TABLE_ID': best_match_row['id'],
            'Spotify_Song_Title': best_match_row['song_title'],
            'Spotify_Artist': best_match_row['song_album_id'],  # Note: Adjust if there's a more direct artist name column
            'MusicBrainz_Song_Title': mb_row['SONGTITLE'],
            'MusicBrainz_Artist': mb_row['MAINARTIST']
        }, ignore_index=True)

# Output the matched DataFrame to CSV
matched_df.to_csv('linked_songs.csv', sep=';', index=False)

print("Linked songs CSV generated.")

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\contributors_non-normalized.csv')

# Select only the 'artist_credit_name' and 'artist_credit_id' columns
df_selected = df[['artist_credit_name', 'artist_credit_id']]

# Drop duplicates based on 'artist_credit_id' to ensure each ID is unique
df_unique = df_selected.drop_duplicates(subset=['artist_credit_id'])

# Save the result to a new CSV file
df_unique.to_csv('unique_artist_credits.csv', index=False)

print('Unique artist credits CSV file has been saved as unique_artist_credits.csv.')


In [ ]:
import pandas as pd

def parse_manual(array_str):
    # Manually parse the string to extract elements between curly braces
    # Remove leading and trailing braces and split by comma
    items = array_str.strip('{}').split(',')
    # Strip quotes and extra spaces from each item
    items = [item.strip('"').strip() for item in items]
    return items

# Load the CSV file
df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\contributors_non-normalized.csv')

# Prepare a list to collect all artist name-MBID pairs
artist_pairs = []
i=0

# Iterate over the DataFrame rows
for _, row in df.iterrows():
    # Parse the 'artist_mbids' and 'individual_artist_names' fields manually
    artist_mbids = parse_manual(row['artist_mbids'])
    individual_artist_names = parse_manual(row['individual_artist_names'])
    
    # Ensure we have equal lengths of MBIDs and names before proceeding
    if len(artist_mbids) == len(individual_artist_names):
        # Pair each individual artist name with its corresponding MBID
        for artist_name, artist_mbid in zip(individual_artist_names, artist_mbids):
            artist_pairs.append((artist_name, artist_mbid))
    else:
        print("Warning: Mismatched MBIDs and artist names for a row, skipping.")
        print(f"Row index: {i}")
        i = i+1

# Convert the list of pairs into a DataFrame, ensuring uniqueness
df_pairs = pd.DataFrame(list(set(artist_pairs)), columns=['individual_artist_name', 'artist_mbid'])

# Save the DataFrame to a new CSV file
df_pairs.to_csv('artist_name_mbid_pairs.csv', index=False)

print('Artist name-MBID pairs CSV file has been saved as artist_name_mbid_pairs.csv.')



In [ ]:
import pandas as pd

# Load the MBID CSV
mbid_df = pd.read_csv('C:\\Users\\Music\\team_project\\team37\\prepopulationStuff\\PythonNotebooksForPrepopulation\\artist_name_mbid_pairs.csv')

# Load the Spotify artist CSV, remember to use the ';' delimiter
spotify_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\artists_data.csv', delimiter=';')

# Perform a direct merge based on artist names
# Note: This assumes 'artist_name' in spotify_df exactly matches 'individual_artist_name' in mbid_df
merged_df = pd.merge(spotify_df, mbid_df, how='left', left_on='artist_name', right_on='individual_artist_name')

# Drop the 'individual_artist_name' column as it's redundant after merge
merged_df.drop(columns=['individual_artist_name'], inplace=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('spotify_artist_with_mbid_direct_match.csv', index=False)

print('Spotify artist data with direct match MBIDs has been saved as spotify_artist_with_mbid_direct_match.csv.')


In [ ]:
import pandas as pd

# Adjust the path to your Spotify artists CSV file
spotify_csv_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\artists_data.csv'
spotify_df = pd.read_csv(spotify_csv_path, delimiter=';')

# Print column names to verify
print(spotify_df.columns)


In [ ]:
import pandas as pd
from datetime import datetime

# Load the CSV file
csv_file_path = 'C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\config\\liquibase\\fake-data\\artists_table_new_notworking.csv'
df = pd.read_csv(csv_file_path, delimiter=';')

# Function to check for empty values in columns that shouldn't have them
def check_empty_values(df, columns):
    for column in columns:
        if df[column].isnull().any():
            print(f"Empty values found in column: {column}")
        else:
            print(f"No empty values in column: {column}")

# Function to check data types
def check_data_types(df):
    errors = []
    for column, expected_type in expected_column_types.items():
        if expected_type == 'numeric':
            if not pd.api.types.is_numeric_dtype(df[column]):
                errors.append(f"Column {column} is not of type {expected_type}")
        elif expected_type == 'string':
            if not pd.api.types.is_string_dtype(df[column]):
                errors.append(f"Column {column} is not of type {expected_type}")
        elif expected_type == 'date':
            try:
                pd.to_datetime(df[column])
            except ValueError:
                errors.append(f"Column {column} contains non-date values")
    
    if errors:
        for error in errors:
            print(error)
    else:
        print("All columns match their expected data types.")

# Columns expected not to have empty values based on your schema (all of them in this case)
mandatory_columns = [
    'id', 'artist_spotify_id', 'artist_name', 'artist_popularity',
    'artist_image_small', 'artist_image_medium', 'artist_image_large',
    'artist_followers', 'date_added_to_db', 'date_last_modified'
]

# Expected data types
expected_column_types = {
    'id': 'numeric',
    'artist_spotify_id': 'string',
    'artist_name': 'string',
    'artist_popularity': 'numeric',
    'artist_image_small': 'string',
    'artist_image_medium': 'string',
    'artist_image_large': 'string',
    'artist_followers': 'numeric',
    'date_added_to_db': 'date',
    'date_last_modified': 'date'
}

# Perform checks
check_empty_values(df, mandatory_columns)
check_data_types(df)


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\album_data_full.csv', sep=';')  # Adjust the path and separator as needed

# Convert the dates in the last two columns
# Assuming the last two columns contain your dates
df.iloc[:, -4] = pd.to_datetime(df.iloc[:, -4], dayfirst=True).dt.strftime('%Y-%m-%d')
df.iloc[:, -3] = pd.to_datetime(df.iloc[:, -3], dayfirst=True).dt.strftime('%Y-%m-%d')

# Save the modified DataFrame back to a new CSV file
df.to_csv('path_to_modified_csv.csv', index=False, sep=';')  # Adjust the path and separator as needed


In [ ]:
import pandas as pd

# Define the path to your CSV file
csv_file_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\album_data_full.csv'

# Load the CSV file
df = pd.read_csv(csv_file_path, delimiter=';')

# Function to check for empty values in mandatory columns
def check_empty_values(df, columns):
    for column in columns:
        if df[column].isnull().any():
            print(f"Empty values found in column: {column}")
        else:
            print(f"No empty values in column: {column}")

# Function to verify data types
def check_data_types(df, column_types):
    for column, expected_type in column_types.items():
        if expected_type == 'numeric':
            if not pd.api.types.is_numeric_dtype(df[column]):
                print(f"Column {column} is not of numeric type.")
            else:
                print(f"Column {column} is of correct numeric type.")
        elif expected_type == 'string':
            if not pd.api.types.is_string_dtype(df[column]):
                print(f"Column {column} is not of string type.")
            else:
                print(f"Column {column} is of correct string type.")
        elif expected_type == 'date':
            try:
                pd.to_datetime(df[column])
                print(f"Column {column} is of correct date type.")
            except ValueError:
                print(f"Column {column} contains incorrect date format.")

# Mandatory columns (not nullable)
mandatory_columns = [
    'id', 'album_spotify_id', 'album_name', 'album_cover_art',
    'album_release_date', 'release_date_precision', 'album_popularity',
    'album_type', 'date_added_to_db', 'date_last_modified', 'musicbrainz_metadata_added'
]

# Columns with specific data types
column_types = {
    'id': 'numeric',
    'album_spotify_id': 'string',
    'album_name': 'string',
    'album_cover_art': 'string',
    'album_release_date': 'date',
    'release_date_precision': 'string',
    'album_popularity': 'numeric',
    'album_type': 'string',
    'spotify_album_upc': 'string',  # Nullable
    'spotify_album_ean': 'string',  # Nullable
    'spotify_album_isrc': 'string',  # Nullable
    'date_added_to_db': 'date',
    'date_last_modified': 'date',
    'musicbrainz_metadata_added': 'boolean',  # Assuming true/false representation
    'musicbrainz_id': 'string'  # Nullable
}

# Check for empty values in mandatory columns
check_empty_values(df, mandatory_columns)

# Check if data types are correct
check_data_types(df, column_types)
31045;59


In [ ]:
import pandas as pd

# Define the path to your CSV file
csv_file_path = 'C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\album_table.csv'

# Load the CSV file
df = pd.read_csv(csv_file_path, delimiter=';')

# Function to check for empty values in mandatory columns
def check_empty_values(df, columns):
    for column in columns:
        if df[column].isnull().any():
            print(f"Empty values found in column: {column}")
        else:
            print(f"No empty values in column: {column}")

# Function to verify data types and print incorrect date formats
def check_data_types_and_dates(df, column_types):
    for column, expected_type in column_types.items():
        if expected_type == 'numeric':
            if not pd.api.types.is_numeric_dtype(df[column]):
                print(f"Column {column} is not of numeric type.")
            else:
                print(f"Column {column} is of correct numeric type.")
        elif expected_type == 'string':
            if not pd.api.types.is_string_dtype(df[column]):
                print(f"Column {column} is not of string type.")
            else:
                print(f"Column {column} is of correct string type.")
        elif expected_type == 'date':
            incorrect_format = df[column].apply(lambda x: check_date_format(x))
            if incorrect_format.any():
                print(f"Rows with incorrect date format in column '{column}':")
                print(df[incorrect_format])
            else:
                print(f"Column {column} is of correct date type.")

# Helper function to check date format
def check_date_format(date_string):
    try:
        pd.to_datetime(date_string, errors='raise')
        return False  # Date format is correct
    except ValueError:
        return True  # Date format is incorrect

# Mandatory columns (not nullable)
mandatory_columns = [
    'id', 'album_spotify_id', 'album_name', 'album_cover_art',
    'album_release_date', 'release_date_precision', 'album_popularity',
    'album_type', 'date_added_to_db', 'date_last_modified', 'musicbrainz_metadata_added'
]

# Columns with specific data types
column_types = {
    'id': 'numeric',
    'album_spotify_id': 'string',
    'album_name': 'string',
    'album_cover_art': 'string',
    'album_release_date': 'date',
    'release_date_precision': 'string',
    'album_popularity': 'numeric',
    'album_type': 'string',
    'spotify_album_upc': 'string',  # Nullable
    'spotify_album_ean': 'string',  # Nullable
    'spotify_album_isrc': 'string',  # Nullable
    'date_added_to_db': 'date',
    'date_last_modified': 'date',
    'musicbrainz_metadata_added': 'boolean',  # Assuming true/false representation
    'musicbrainz_id': 'string'  # Nullable
}

# Check for empty values in mandatory columns
check_empty_values(df, mandatory_columns)

# Check if data types are correct and for incorrect date formats
check_data_types_and_dates(df, column_types)


In [ ]:
import pandas as pd

# Load the CSV file
csv_file_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\tracks_data_full.csv'  # Update this to the path of your CSV file

# Load the CSV file
df = pd.read_csv(csv_file_path, delimiter=';')  # Ensure delimiter matches your CSV format

# Function to check for empty values in mandatory columns
def check_empty_values(df, columns):
    for column in columns:
        if df[column].isnull().any():
            print(f"Empty values found in column: {column}")
        else:
            print(f"No empty values in column: {column}")

# Function to verify data types
def check_data_types(df, column_types):
    for column, expected_type in column_types.items():
        if expected_type == 'numeric':
            if not pd.api.types.is_numeric_dtype(df[column]):
                print(f"Column {column} is not of numeric type.")
            else:
                print(f"Column {column} is of correct numeric type.")
        elif expected_type == 'string':
            if not pd.api.types.is_string_dtype(df[column]):
                print(f"Column {column} is not of string type.")
            else:
                print(f"Column {column} is of correct string type.")
        elif expected_type == 'date':
            try:
                pd.to_datetime(df[column])
                print(f"Column {column} is of correct date type.")
            except ValueError:
                print(f"Column {column} contains incorrect date format.")
        elif expected_type == 'boolean':
            # Checking if the column is boolean; Assuming boolean is represented as True/False or 1/0
            if not pd.api.types.is_bool_dtype(df[column]) and not all(df[column].dropna().isin([0, 1, 'True', 'False', True, False])):
                print(f"Column {column} is not of boolean type.")
            else:
                print(f"Column {column} is of correct boolean type.")

# Mandatory columns (not nullable)
mandatory_columns = [
    'id', 'song_spotify_id', 'song_title', 'song_duration', 
    'song_album_type', 'song_album_id', 'song_explicit', 
    'song_popularity', 'song_track_features_added', 'song_date_added_to_db', 
    'song_date_last_modified'
]

# Columns with specific data types
column_types = {
    'id': 'numeric',
    'song_spotify_id': 'string',
    'song_title': 'string',
    'song_duration': 'numeric',
    'song_album_type': 'string',
    'song_album_id': 'string',
    'song_explicit': 'boolean',
    'song_popularity': 'numeric',
    'song_preview_url': 'string',  # Nullable
    'song_track_features_added': 'boolean',
    # Assuming 'floatType' corresponds to 'numeric' in Python/Pandas
    'song_acousticness': 'numeric',  # Nullable
    'song_danceability': 'numeric',  # Nullable
    'song_energy': 'numeric',  # Nullable
    'song_instrumentalness': 'numeric',  # Nullable
    'song_liveness': 'numeric',  # Nullable
    'song_loudness': 'numeric',  # Nullable
    'song_speechiness': 'numeric',  # Nullable
    'song_tempo': 'numeric',  # Nullable
    'song_valence': 'numeric',  # Nullable
    'song_key': 'numeric',  # Nullable
    'song_time_signature': 'numeric',  # Nullable
    'song_date_added_to_db': 'date',
    'song_date_last_modified': 'date',
    'album_id': 'numeric'  # Nullable
}

# Check for empty values in mandatory columns
check_empty_values(df, mandatory_columns)

# Check if data types are correct
check_data_types(df, column_types)


In [ ]:
import pandas as pd

# Adjust these settings as needed
pd.set_option('display.max_rows', None)  # This will allow all rows to be displayed
pd.set_option('display.max_columns', None)  # This will allow all columns to be displayed
pd.set_option('display.width', 1000)  # Adjust the width to accommodate the number of columns
pd.set_option('display.max_colwidth', None)  # This ensures that the content of each column is fully displayed

# Define the path to your CSV file
csv_file_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\tracks_data_full.csv'  # Update this to the path of your CSV file

# Load the CSV file
df = pd.read_csv(csv_file_path, delimiter=';')

# Function to print rows with empty values in a specific column
def print_rows_with_empty_values_in_column(df, column_name):
    empty_rows = df[df[column_name].isnull() | (df[column_name] == '')]
    if not empty_rows.empty:
        print(f"Rows with empty values in column '{column_name}':")
        # print the song_spotfiy_id for each row with empty values
        print(empty_rows['song_spotify_id'])
    else:
        print(f"No empty values found in column '{column_name}'.")

# Print all rows where the song_title is empty
print_rows_with_empty_values_in_column(df, 'song_title')



In [ ]:
import pandas as pd
import os
import json

# Load the CSV files
albums_df = pd.read_csv('C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\album_table.csv', delimiter=';', dtype={'id': str, 'album_spotify_id': str})
artists_df = pd.read_csv('C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\artists_table.csv', delimiter=';', dtype={'id': str, 'artist_spotify_id': str})

# Create dictionaries for ID mappings
album_id_map = albums_df.set_index('album_spotify_id')['id'].to_dict()
artist_id_map = artists_df.set_index('artist_spotify_id')['id'].to_dict()

artist_album_mapping = []

# Assuming the JSON structure is as shown in your example
for root, dirs, files in os.walk('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\ALBUMS'):
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                data = json.load(f)
                for album in data['albums']:
                    album_id = album_id_map.get(album['id'])
                    if not album_id:
                        print(f"Album ID {album['id']} not found in album CSV.")
                        continue
                    for artist in album['artists']:
                        artist_id = artist_id_map.get(artist['id'])
                        if not artist_id:
                            print(f"Artist ID {artist['id']} not found in artist CSV.")
                            continue
                        artist_album_mapping.append({'artistID': artist_id, 'albumID': album_id})

# Create DataFrame and remove duplicates
artist_album_df = pd.DataFrame(artist_album_mapping).drop_duplicates()

# Save the DataFrame to CSV
artist_album_df.to_csv('artist_album_mappings_new.csv', index=False, sep=';')


In [ ]:
import pandas as pd
import os
import json

# Load the CSV files
albums_df = pd.read_csv('C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\album_table.csv', delimiter=';', dtype={'id': str, 'album_spotify_id': str})
artists_df = pd.read_csv('C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\artists_table.csv', delimiter=';', dtype={'id': str, 'artist_spotify_id': str})

# Create dictionaries for ID mappings
album_id_map = albums_df.set_index('album_spotify_id')['id'].to_dict()
artist_id_map = artists_df.set_index('artist_spotify_id')['id'].to_dict()

artist_album_mapping = []
missing_artists = {}  # Store missing artist IDs and the album IDs they're attributed to

# Assuming the JSON structure is as shown in your example
for root, dirs, files in os.walk('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\ALBUMS'):
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                data = json.load(f)
                for album in data['albums']:
                    album_id = album_id_map.get(album['id'])
                    if not album_id:
                        print(f"Album ID {album['id']} not found in album CSV.")
                        continue
                    for artist in album['artists']:
                        artist_id = artist_id_map.get(artist['id'])
                        if not artist_id:
                            # Record the missing artist ID along with the current album ID
                            missing_artists.setdefault(artist['id'], []).append(album['id'])
                            print(f"Artist ID {artist['id']} not found in artist CSV, attributed to Album ID {album['id']}.")
                            continue
                        artist_album_mapping.append({'artistID': artist_id, 'albumID': album_id})

# Display missing artist IDs and the albums they're attributed to
for artist_id, album_ids in missing_artists.items():
    print(f"Artist ID {artist_id} (not found) is attributed to Album IDs: {', '.join(album_ids)}")


In [ ]:
import pandas as pd

# Load the tracks CSV file
csv_file_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\tracks_data_full.csv'
df = pd.read_csv(csv_file_path, delimiter=';', dtype=str)

# Add a new column `recording_mbid` with empty values (or any default value you prefer)
# The column is added at the specific position before `album_id`
df.insert(df.columns.get_loc("album_id"), "recording_mbid", "")

# List of song_spotify_id to delete
ids_to_delete = [
    "75UvH3TDraHQon52VexYrd",
    "72QOdqmnjwpx4fJcUhjjLp",
    "1dF3LpNoeDV25vFEucwqxL",
    "74T4vgexWpBwybtGx0KbM7",
    "3JBcNHwpW8WcJMnXLXyrgK",
    "3LUCdcXCYko2PsbMUCOLZm",
    "26VIoWSeLRLh8yI3PuWQ12",
    "2yUpba8YCjUhsvqUszidRA",
    "2EDuAbx8M1pK0xumWpCPV0",
    "4CBsZXLknCBL2Yx1oHtEd1",
    "4FKYbGJ8ZWwqY6BzRGunp3",
    "74wJH625TY4DsGIUlEU8N2",
    "5OVSK2zk2PWiWUo1AKNPHb",
    "4ZsZvWhziGDL4RIslCnTSg",
    "7xv6Eu2FvAYnhnVZ9oNDy9",
    "4uxqGutcmbvpdxqploz0nP",
    "0c2OsDg3NUnbB81Zcdv3hO",
    "3CWtEob3yRj8kxvmtERogB",
    "2Lk7JJX3c6guZ7Zz1kXZee",
    "1Ak80nlobVpr3j1eVhDin2",
    "2PiZCczBPG6B2ImYpou6Af",
    "2OvCbJTgPJYh4gndgTOVQu",
    "7bszJWvC2o9Ux72fWipYJL",
    "7jNOZY1aOlm3moWVG8WoKi",
    "5b9nPvneKlpS62phraSwYV",
    "7zYQM45B7J9NKKxG1if4Wz",
    "4L264W9L6K78M8QnBXPz03",
    "456EsNmqInGDaX0qxgfBnX",
    "3gEhgYimmqodF6BrvlKtar",
    "1qqxdGz5o6Kfh1ZyvRTZvi",
    "62LpvA7MNOuDdhIW6Kh9AI",
    "78Wt8QOMTBHfvn1E57b5Xu",
    "1TmmI3VPZjMwsqV7kMskwx",
    "0iW4yp45NhZHmuw4d9uNF6",
    "47PqaO65CrVnMwH7CfxJvb",
    "6uepvILocapt6Fg40WJS99",
    "5MIxwk6y73xTDqGj4hBbkz",
    "7ahK0JIHV3n7SWja1Fmi4b",
    "5PdyHsV2Av4ih0jnsguG1m",
    "0k5jqDnVF8i4GdnDqabodl",
    "25DzjQxLNatPVUzo0rC7AW",
    "1zcR9YAcOzZ5WmBBQpZpnY",
    "2GWruQDCJeeNSaOlV8mgvv",
    "0xUMokwO3lo0YkdAfQySjT",
    "3wcDi6kWtAetFydVN4Bbrx",
    "7l67g6KzQsFBDPSKvyPsAy",
    "0dsTq154TF7mCMg18lpI58",
    "5gSiQViEHnsmGuQ5Tibvs5",
    "6zMpg17rh0XftHphBJJmmd",
    "2njlNRtphrdtdHREmrEqR3",
    "3NXYmpj0lp5427XbKbUcfC",
    "6JbXjioNWKx6XjBcPleJ72",
    "2JPJpFF275xAB8H95o8ueG",
    "2QigB5AHMSzX6X82ijLosf",
    "67TrkSnOD9Nui8m95Qe5q2",
    "2wT3CYlhPHfj4eh2X06TbX",
    "2V2TFejf9JlzXVpHIr6jSt",
    "2trNJPmMIgDeowTdZRRP3F",
    "12Xx9dJ5oHW0MwyzYOtur7",
    "4M6XTI8Uk7hMAs2UdEkwRl",
    "33J6EyMfqdhamFdTuAoJX7",
    "1UL8YBCLVVnDzmZeoV5ttL",
    "3Zi0Mr2fWJsf41fTvQlH9U",
    "3AdrYdf4H4z0AT93tGmKEf",
    "0tpdtfsYH4GL2iYWP6XujW",
    "5EbuaL6OikuBKntP1rOeY1",
    "54OE3k8dzodj9Dkvzq6hOD",
    "5wQHfM4ImDZAfb20sZED1Y",
    "5IW9oM2x5mNfReN28qNDAo",
    "2xhMF0vhmuI2utvwpbLoLU",
    "64cRwHXDTKmcwcBgG8jKcP",
    "0JCfef9cI9aHjUuV542gAp",
    "0jF41wuIItYGsK0xb72h6e",
    "3b30Jt6Jv0AVOGy7knAyEd",
    "4NFtHPM5C74PE4me5EAzjf",
    "2fp46neVxgQdC9WYfskdU8",
    "47mweMIEiVG1VTURjooVdU",
    "0mGGQ9BKBsGVZURwtC1ri6",
    "4RLGpC5pvisS0py1wJ8OF4",
    "2b42gCsSo7Kwg1MgOTXlv8",
    "3HihOJceOThzpsbAFCLq9p",
    "4fguT5jLuyS6aIszCjkoUW",
    "2XlGPrt6URuTsuwXCqQoXp",
    "3qgvtszdPCoKBj02aflFjM",
    "5nGitkFcddduRwEFCQAmaY",
    "0a99u1uV6YsgoZJ0ArL8YG",
    "17fEyUuqKTEHgh6eGijMNd",
    "5YRyBSPSXi1kha3p4jnW30",
    "67HIfSjDNBiu4L4KHe4Iwu",
    "5kax80HlqMnyxhxxzvrHIg",
    "3bkrlUkcpVUaFfoF8aUsMG",
    "7H12MyuGjh83lr30LZDemx",
    "7Gv17UJTle9X61IPSAx60k",
    "3ymrE7n8eJiXEOdFJ2lplT",
    "7pt1yDbKBo3M0cwVeVSqYp",
    "7EgwllELTl4RMOITMuhIa3",
    "65qooAhuNUPKG3AYNQx3hQ",
    "3iIXnGdigJTNZRkUbNVZVQ",
    "72HJ0dXhrCp2h1tUCUbwP6",
    "1Oo1O7C2yLwKx7YF7EjaJY",
    "45Q19MuCEgo3ebzcmmQJeA",
    "5lwCBdknMtWs4OKlYEdb5w",
    "1Gnmu7Li3aJ1IAh3K5V2Fl",
    "60224pftm64rJrayNN2MTA",
    "4siKkiCT4VP6InhBOHV4vP",
    "4hWiD8q9m4gs9QlqydcEv7",
    "5yyG2O9ABwlEuBunYWmWT0",
    "4NyRcqUXDCRxzWqZmZxyWl",
    "6xIkjw9xcd8s60dLUtAxM6",
    "0fhZjHAfWzfwfkMW9Zdpga",
    "4doiaWc4GbuOpIT9GeVwub",
    "4lp6ZxoXb6BU3emooKHcNe",
    "0U7ruXdI2rjw14XB0yabUc",
    "2xJuT0ZJz0WoIYGMKXG01t",
    "4k86QF2GSQh7wGVb2CWzSL",
    "0RskFK29CBwj1GpkpqaYEG",
    "1dxUsz7reRhvjAfhja9w0d",
    "2vN2OF6zeAIYmGBjId0g5r",
    "4bZ6YhwSfSb3kLtR8F0R5P",
    "4KKMeWvjZqvXRnPpE83GrC"
]
# Delete rows with the specified song_spotify_id
df = df[~df['song_spotify_id'].isin(ids_to_delete)]

# Save the updated DataFrame back to CSV
df.to_csv('updated_tracks_data_full.csv', index=False, sep=';')


In [ ]:
import pandas as pd

# Load the tracks and album CSVs into DataFrames
tracks_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\tracks_data_full.csv', delimiter=';', dtype=str)
albums_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\album_data_full.csv', delimiter=';', dtype=str)

# Create a mapping dictionary from the album CSV: Spotify ID -> album_id
spotify_id_to_album_id = dict(zip(albums_df['album_spotify_id'], albums_df['id']))

# Use the map to update the album_id in the tracks DataFrame
# The `song_album_id` column contains the Spotify IDs that will be used for mapping
tracks_df['album_id'] = tracks_df['song_album_id'].map(spotify_id_to_album_id)

# Save the updated DataFrame back to CSV
tracks_df.to_csv('updated_tracks_data_full.csv', index=False, sep=';')


In [ ]:
import pandas as pd

# Load the album CSV into a DataFrame
album_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\album_data_full.csv', delimiter=';', dtype=str)

# Drop the 'URL' column
album_df = album_df.drop(columns=['URL'])

# Save the updated DataFrame back to CSV
album_df.to_csv('updated_album_data_full.csv', index=False, sep=';')


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\config\\liquibase\\fake-data\\artists_table_new_notworking.csv', sep=';')

# Define a function to convert date format from DD/MM/YYYY to DD-MM-YYYY
def convert_date_format(date_str):
    return pd.to_datetime(date_str, dayfirst=True).strftime('%d-%m-%Y')

# Apply the function to the 'date_added_to_db' and 'date_last_modified' columns
df['date_added_to_db'] = df['date_added_to_db'].apply(convert_date_format)
df['date_last_modified'] = df['date_last_modified'].apply(convert_date_format)

# Convert artist_followers from float to int by first filling NaN values to avoid errors
df['artist_followers'] = df['artist_followers'].fillna(0).astype(int)

# Save the modified DataFrame back to a CSV file
df.to_csv('modified_file.csv', sep=';', index=False)

# Display the head of the modified DataFrame as a sanity check
print(df.head())



In [ ]:
import pandas as pd

def convert_to_iso_format(date_str):
    """
    Convert dates from DD-MM-YYYY to YYYY-MM-DD format.
    """
    return pd.to_datetime(date_str, dayfirst=True).strftime('%Y-%m-%d')

# Load your CSV file
df = pd.read_csv('C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\config\\liquibase\\fake-data\\artists_table_new_notworking.csv', sep=';')


# Convert 'date_added_to_db' and 'date_last_modified' columns to the ISO 8601 date format (YYYY-MM-DD)
df['date_added_to_db'] = df['date_added_to_db'].apply(convert_to_iso_format)
df['date_last_modified'] = df['date_last_modified'].apply(convert_to_iso_format)

# Convert 'artist_followers' from float to int, handling NaN values by converting them to 0
df['artist_followers'] = df['artist_followers'].fillna(0).astype(int)

# Save the modified DataFrame back to a new CSV file
modified_csv_file_path = 'modified_file.csv'  # You can change this to your desired output file path
df.to_csv(modified_csv_file_path, sep=';', index=False)

print("File has been modified and saved as:", modified_csv_file_path)


In [ ]:
import pandas as pd

# Path to your CSV file
csv_file_path = 'C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\album_table.csv'
# Path for the output CSV file with rows having null album_release_date
output_csv_file_path = 'null_release_dates.csv'

def check_null_release_dates(csv_path):
    # Read the CSV file
    df = pd.read_csv(csv_path, sep=';')

    # Filter rows where album_release_date is null or empty
    null_release_dates = df[df['album_release_date'].isnull() | (df['album_release_date'] == '')]

    # Check if there are any such rows
    if not null_release_dates.empty:
        print("Rows with null or empty album_release_date:")
        print(null_release_dates)

        # Optional: Save these rows to a new CSV file
        null_release_dates.to_csv(output_csv_file_path, sep=';', index=False)
        print(f"Saved rows with null or empty album_release_date to {output_csv_file_path}")
    else:
        print("No rows with null or empty album_release_date found.")

# Call the function with your CSV file path
check_null_release_dates(csv_file_path)


In [ ]:
import pandas as pd

# Path to your CSV file
csv_file_path = 'C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\album_table.csv'
# Path for the output CSV file with adjusted release dates
output_csv_file_path = 'adjusted_release_dates.csv'

def adjust_release_date(row):
    # Check the release_date_precision and adjust album_release_date accordingly
    if row['release_date_precision'] == 'year':
        # For year precision, add "-01-01"
        return f"{row['album_release_date']}-01-01"
    elif row['release_date_precision'] == 'month':
        # For month precision, add "-01"
        return f"{row['album_release_date']}-01"
    else:
        # If it's already precise, or another value, leave it as is
        return row['album_release_date']

def process_csv_file(csv_path, output_path):
    # Load the CSV file
    df = pd.read_csv(csv_path, sep=';')
    
    # Apply the adjustment function to each row
    df['album_release_date'] = df.apply(adjust_release_date, axis=1)
    
    # Save the modified DataFrame to a new CSV file
    df.to_csv(output_path, sep=';', index=False)
    
    print(f"Processed file saved to: {output_path}")

# Call the function with your CSV file path
process_csv_file(csv_file_path, output_csv_file_path)


In [ ]:
import pandas as pd
import math
import os

# Path to the large CSV file
input_csv_path = 'C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\tracks_data_full_removed.csv'

# Output directory for the smaller CSV files
output_dir = 'split_csv_files'

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Separator character used in the CSV file
separator = ';'

# Load the large CSV file
df = pd.read_csv(input_csv_path, sep=separator)

# Number of rows in each split file
rows_per_file = 400000

# Total number of rows in the DataFrame
total_rows = len(df)

# Calculate the number of files needed
num_files = math.ceil(total_rows / rows_per_file)

# Split and save the DataFrame into smaller CSV files
for i in range(num_files):
    start_row = i * rows_per_file
    end_row = min((i + 1) * rows_per_file, total_rows)
    df_subset = df.iloc[start_row:end_row]
    output_file_path = os.path.join(output_dir, f'split_file_{i + 1}.csv')
    df_subset.to_csv(output_file_path, sep=separator, index=False)

print(f"Split completed. {num_files} files were created in '{output_dir}' directory.")


In [ ]:
import pandas as pd

# Load the CSV files into Pandas DataFrames
songs_df = pd.read_csv('C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\tracks_data_full_removed.csv', sep=';')
albums_df = pd.read_csv('C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\album_table.csv', sep=';')

# Extract the album_id columns from both DataFrames
song_album_ids = songs_df['album_id'].unique()
album_ids = albums_df['id'].unique()

# Find album_ids in the song CSV that are not in the album CSV
missing_album_ids = set(song_album_ids) - set(album_ids)

# Initialize an empty list to hold the IDs of songs with missing album_ids
missing_songs = []

# Check each song to see if its album_id is missing in the album CSV
for index, row in songs_df.iterrows():
    if row['album_id'] in missing_album_ids:
        missing_songs.append(row['id'])

if missing_songs:
    print("The following song IDs reference album_ids that are missing in the albums CSV:")
    print(missing_songs)
else:
    print("All songs in the songs CSV have corresponding album_ids in the albums CSV.")

In [ ]:
import pandas as pd

# Path to the original tracks CSV file
input_csv_path = 'C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\tracks_data_full.csv'
# Path for the output CSV file without the specified songs
output_csv_path = 'filtered_tracks.csv'

# List of song IDs to remove
song_ids_to_remove = [
    62034, 88913, 112688, 300187, 316102, 329999, 367946, 375943, 386729, 401892,
    435595, 436765, 447794, 447852, 608310, 711292, 790732, 883084, 920213, 1018358,
    1114764, 1127693, 1142259, 1168089, 1187599, 1265380, 1320394, 1402381, 1413819,
    1512351, 1563201, 1632407, 1681835, 1730948, 1770887, 1791508, 1808512, 1851036,
    1889088, 1941659, 1949441, 1982776, 2034044, 2109511, 2111128, 2147642, 2194426,
    2201476, 2211371, 2320285, 2327313, 2361529, 2363631, 2406470, 2437122, 2505740,
    2513991, 2624535, 2638046, 2772894, 2777596, 2789005, 2818861, 2819844, 2883260,
    2907509, 2917805, 2954339, 3170871, 3329632, 3371496, 3460982, 3496147, 3625644,
    3628044, 3650231, 3658670, 3673094, 3701507
]

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv(input_csv_path, sep=';')

# Filter the DataFrame to exclude rows with the specified song IDs
filtered_df = df[~df['id'].isin(song_ids_to_remove)]

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv(output_csv_path, sep=';', index=False)

print(f"Filtered CSV saved to '{output_csv_path}'.")


In [ ]:
import pandas as pd

# Path to your CSV file
csv_file_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\WORKING\\spotify_musicbrainz_matches_with_fuzzy_album_only.csv'
# Path for the output CSV file
output_csv_path = 'filtered_no_duplicates.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Sort the DataFrame by spotify_id and match_score in descending order
df_sorted = df.sort_values(by=['spotify_id', 'match_score'], ascending=[True, False])

# Drop duplicates, keeping the first occurrence (which has the highest match_score due to the sort)
df_filtered = df_sorted.drop_duplicates(subset='spotify_id', keep='first')

# Save the filtered DataFrame to a new CSV file
df_filtered.to_csv(output_csv_path, index=False)

print(f"Filtered CSV without duplicates saved to '{output_csv_path}'.")


In [ ]:
import pandas as pd

# Paths to your CSV files
csv_file_path_1 = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\WORKING\\spotify_musicbrainz_matches_filtered_no_duplicates.csv'  # Update this to the path of your first CSV
csv_file_path_2 = 'C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\artists_table.csv'  # Update this to the path of your second CSV

# Output path for the merged CSV file
output_csv_path = 'merged_csv_with_musicbrainz_id.csv'

# Load the CSV files into DataFrames
df1 = pd.read_csv(csv_file_path_1)
df2 = pd.read_csv(csv_file_path_2, sep=';')

# We only need the 'spotify_id' and 'musicbrainz_id' columns from the first DataFrame
df1_relevant = df1[['spotify_id', 'musicbrainz_id']]

# Rename columns in df1_relevant for consistency with df2
df1_relevant.rename(columns={'spotify_id': 'artist_spotify_id'}, inplace=True)

# Merge df2 with df1_relevant to add the musicbrainz_id based on artist_spotify_id
# Perform a left join to keep all rows from df2 and only add musicbrainz_id where matches are found
merged_df = pd.merge(df2, df1_relevant, on='artist_spotify_id', how='left')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv(output_csv_path, sep=';', index=False)

print(f"Merged CSV saved to '{output_csv_path}'.")


In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
csv_file_path = 'C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\artists_table.csv'  # Update this to the path of your CSV file
df = pd.read_csv(csv_file_path, sep=';')

# Merge the musicbrainz_id_x and musicbrainz_id_y columns
# Assuming we prioritize non-empty values in musicbrainz_id_x
df['musicbrainz_id'] = df['musicbrainz_id_x'].where(df['musicbrainz_id_x'].notna(), df['musicbrainz_id_y'])

# Drop the original musicbrainz_id_x and musicbrainz_id_y columns
df.drop(['musicbrainz_id_x', 'musicbrainz_id_y'], axis=1, inplace=True)

# Save the updated DataFrame back to a CSV file
output_csv_path = 'updated_csv_file.csv'  # Update this to your desired output file path
df.to_csv(output_csv_path, sep=';', index=False)

print(f"Updated CSV saved to '{output_csv_path}'.")


In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
csv_file_path = 'C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\artists_table.csv'  # Update this to the path of your CSV file
df = pd.read_csv(csv_file_path, sep=';')

# Find duplicates based on the 'id' column
duplicates = df[df.duplicated('id', keep=False)]  # keep=False marks all duplicates as True

# Check if there are any duplicates
if not duplicates.empty:
    print("Duplicate entries based on 'id':")
    print(duplicates)
    # Optional: Save the duplicates to a new CSV file for further investigation
    duplicates.to_csv('duplicates_in_csv.csv', sep=';', index=False)
    print("Duplicates saved to 'duplicates_in_csv.csv'.")
else:
    print("No duplicate 'id' entries found.")


In [ ]:
mbids_to_remove = ["ee58c59f-8e7f-4430-b8ca-236c4d3745ae",
"614ee114-7d22-4d35-bfa4-6b9ee8292766",
"18bb9c95-5cd1-4aaf-8069-1a216ac7dde4",
"569fc5bf-bcb6-4bc5-9a00-bd0d258e34f1",
"55501cc6-c291-4456-a86a-1ef61019ac95",
"e2459c47-a682-405f-bd20-0105b9a7f50a",
"a40b46f7-7b40-4a28-9dec-a1e3cb440a8b",
"379fb100-063e-4472-b7c2-5bb9eda435ed",
"80cadd99-f560-41e3-babd-16292bbd248a",
"9953ebf6-1836-45ef-bcca-bee2e32e29c6",
"67a71fbe-1a41-4c6d-93ab-09c000ab4aa7",
"e2b34653-c813-4c07-93ad-674bc58e4696",
"b97a49d7-e8b9-406d-9e79-2720004e2806",
"11cb567e-ecac-4226-8944-f862440e8bca",
"408b2a33-d229-46a5-a361-0667bc616e34",
"807dcde7-5deb-4c18-940b-ca3b0dc0e469",
"ed2ac1e9-d51d-4eff-a2c2-85e81abd6360",
"66a3f091-c5a7-4ead-8fd8-1adf4ddccd78",
"7b6c5ade-d762-4456-804b-8321f93a8307",
"ac940ef6-5d4b-4da3-8c3a-245954c299b9",
"21fc9320-c4dd-4b92-9963-927527b4c59e",
"467a0aba-7e76-4080-9643-b88f82452d66",
"5fdae5d2-8aff-4671-8cf8-3b216f63265f",
"7cbdfc61-30a9-4f17-b2fc-dcae3757adba",
"e5750671-59c8-447b-8511-fd29d85596b6",
"a9443494-9d85-4ff5-b53f-da6d912de210",
"6d1d17dc-256b-4ebf-8cad-db2abb60b1e9",
"85e260f5-a1b4-4bb1-bb8a-7d0c8db0588b",
"1dce6682-6db5-495a-afbf-c8566d3fd084",
"9f0a5371-5a2b-49bc-9083-0c93d4571a76",
"714f3d1a-6e6a-46a2-beda-5bcbc37e9a13",
"bf28e2d7-ba14-4578-afb9-e14f92153bad"]


In [ ]:
import pandas as pd

# Load the CSV file
csv_file_path = 'C:\\Users\\Music\\team_project\\team37\\src\\main\\resources\\downloaded_files\\artists_table.csv'  # Update with the path to your CSV file
df = pd.read_csv(csv_file_path, sep=';')

# Filter the DataFrame to remove rows with MBIDs in the mbids_to_remove list
filtered_df = df[~df['musicbrainz_id'].isin(mbids_to_remove)]

# Save the filtered DataFrame to a new CSV file
output_csv_path = 'cleaned_csv_file.csv'  # Update with your desired output path
filtered_df.to_csv(output_csv_path, sep=';', index=False)

print("Filtered CSV saved to:", output_csv_path)
